In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests


In [431]:
URL ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Tilastopaja - 28th SEA Games - 2015.html'  # open a locally saved copy of html
HTMLFileToBeOpened = open(URL, "r") 
contents = HTMLFileToBeOpened.read() 
soup = BeautifulSoup(contents, 'lxml') 


In [432]:
print(soup.body.prettify()) 


<body onload="document.getElementById('home').className='active'; leader(); SetLoginFocus();">
 <script>
  function load(){
	document.getElementById("siteLoader").style.display="block";
}
 </script>
 <div class="wrapper">
  <div class="top">
   <div class="logo">
    <h1>
     <a ref="https://www.tilastopaja.info">
      TILASTOPAJA
     </a>
    </h1>
   </div>
   <div class="container">
    <ul id="nav">
     <li class="haku">
      <div>
       SEARCH FOR AN ATHLETE
       <form action="https://www.tilastopaja.info/db/ats.php" method="GET">
        <input class="text" name="Name" size="20" style="font-weight: normal;"/>
        <input class="button" type="submit" value="Go!"/>
       </form>
      </div>
     </li>
     <li id="home">
      <a href="https://www.tilastopaja.info/">
       Home
      </a>
     </li>
     <li>
      <a href="#s1">
       Free
      </a>
      <span id="s1">
      </span>
      <ul class="subs">
       <li>
        <a href="https://www.tilastopaja.info/

In [438]:
# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A> (name) or <B> (heats and stage)

lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
dict_result2=[]  # length 2 dict
row=[]

temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT'])

h1s = soup.find_all('h1')

for h1 in h1s:
    
    print(h1.text)


for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
        print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
        
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                
                if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           
                    
                    names.append(n.text) # add 'clean' name to list
                              
            for m in value.find_all('b'):  # extract heats and stage
                                
                print('MMMMM', m.text)
                 
          #  for count_b, value_b in enumerate(value.find_all('b')):  # extract the name and put into a list
                                
          #      print('MMMMM', value_b.text)

                                       
            if len(lists)==2:  # captures gender
                
               # heat=''
                wind=''
                dict_result=''
                dict_result2=''
              #  stage=''
                
                print('RESET heat, stage HERE')
                
                if '4 x 100m' not in event and '4 x 400m' not in event:
                    
                    heat = ''
                    stage = ''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
            if 'Date' in value.text:
                date=value.text
                          
            if 'Wind' in value.text:
                wind=value.text
     #           print('WINDDDDD', value.text)
                
            print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            if 'Wind' in lists[2]:
                
                wind = lists[2][5:]  # slice out the 'Wind:'
            
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
            if len(lists[2])>1:
                                
                date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
            date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            print('Reset STAGE, HEAT here')
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
            
                        
        if len(lists)==11: # dict length 10 contains athlete info
                        
            dict_result=lists
                        
            
                        
        #    dict_result2 = [] # reset attempts
            
                        
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
            
                    
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
        print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        try:
            
            if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:
                print('dict_result2', dict_result2)
                
                if len(dict_result2)>0:
                          
                    
                    row.append(dict_result[0]) # rank
             #       for k in range(0, len(names)):  #search and replace name
                        
                    #    hh=''
                    
                   #     if names[k] in dict_result[1]:
                            
                   #         print('XXX', names[k], dict_result[1])
                            
                   #         hh=names[k]

                  #  row.append(hh) # name  (dict_result[1])
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                        row.append(' ')

                 #   row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append('28th Asian Games')
                    row.append('2015')
                    row.append(date)
                    row.append(event)
                    row.append(' ')
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                                        
                else:
                    
                    pass

                    
            
            elif (event and dict_result[3] and dict_result[5]) is not None:
                                    
               #

                row.append(dict_result[0]) # rank
                
                if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                    row.append(names[-1])
                else:
                    row.append(' ')

                
             #   row.append(names[-1]) # name 
                
            #    for k in range(0, len(names)):  #search and replace name
                    
            #        hh=''
                    
            #        if names[k] in dict_result[1]:
                    
            #            hh=names[k]

             #   row.append(hh) # name  (dict_result[1])

               # row.append(dict_result[1]) # name  (dict_result[1])
                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append('28th SEA Games')
                row.append('2015')
                row.append(date)
                row.append(event)
                row.append(' ')
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB                    
                row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT
                
               

            #print(row)
            
        except:
            
            pass
        
        if len(row)>5:
                    
            temp_df.loc[len(temp_df)] = row

        print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
            
            
        
        
    wind=''
    heat=''
    date=''
    
    lists=[]
    dict_result = []
    row=[]
                        
            
                
            
            
            


        
      

TILASTOPAJA
RESULTS: 28th South East Asian Games, Singapore, SGP  - 9 - 12 June 2015 
LATEST MAJOR RESULTS
▼LATEST MAJOR RESULTS
NEW SECTION
TDTD 0 
 Men's events: 


Please choose men's event
Men  100m
Men  200m
Men  400m
Men  800m
Men  1500m
Men  5000m
Men  10,000m
Men  Marathon
Men  3000m Steeplechase
Men  110m Hurdles
Men  400m Hurdles
Men  High Jump
Men  Pole Vault
Men  Long Jump
Men  Triple Jump
Men  Shot Put
Men  Discus Throw
Men  Hammer Throw
Men  Javelin Throw
Men  Decathlon
Men  20km Race Walk
Men  4 x 100m
Men  4 x 400m



TDTD 1      
TDTD 2 
 Women's events: 


Please choose women's event
Women  100m
Women  200m
Women  400m
Women  800m
Women  1500m
Women  5000m
Women  10,000m
Women  Marathon
Women  3000m Steeplechase
Women  100m Hurdles
Women  400m Hurdles
Women  High Jump
Women  Pole Vault
Women  Long Jump
Women  Triple Jump
Women  Shot Put
Women  Discus Throw
Women  Hammer Throw
Women  Javelin Throw
Women  Heptathlon
Women  20km Race Walk
Women  4 x 100m
Women  4 x 400m


In [434]:
len(names)

280

In [435]:
names

['Eric\xa0Cray',
 'Yaspy\xa0Bobi',
 'Abdul Hamid Abdullah\xa0Iswandi',
 'Calvin Kang\xa0Li Loong',
 'Jirapong\xa0Meenapra',
 'Mohd Amiruddin\xa0Jamal',
 'Le Trong Hinh\xa0',
 'Mohd Fakhri\xa0Ismail',
 'Eric\xa0Cray',
 'Jirapong\xa0Meenapra',
 'Abdul Hamid Abdullah\xa0Iswandi',
 'Mohd Amiruddin\xa0Jamal',
 'Mohd Izzuddin\xa0Yahya',
 'Mohd Asy Syafie\xa0Bohari',
 'Yaspy\xa0Bobi',
 'Calvin Kang\xa0Li Loong',
 'Mohd Fakhri\xa0Ismail',
 'Le Trong Hinh\xa0',
 'Nixson\xa0Kennedy',
 'Phearath\xa0Nget',
 'Le Trong Hinh\xa0',
 'Jaran\xa0Sathoengram',
 'Jirapong\xa0Meenapra',
 'Mohd Izzuddin\xa0Yahya',
 'Aravinthevar\xa0Gunasegaran',
 'Mohd\xa0Fadlin',
 'Lee Cheng Wei\xa0',
 '\xa0Sapwaturrahman',
 'Le Trong Hinh\xa0',
 'Jirapong\xa0Meenapra',
 'Mohd\xa0Fadlin',
 'Mohd Izzuddin\xa0Yahya',
 'Aravinthevar\xa0Gunasegaran',
 'Jaran\xa0Sathoengram',
 'Lee Cheng Wei\xa0',
 '\xa0Sapwaturrahman',
 'Phearath\xa0Nget',
 'Kunanon\xa0Sukkaew',
 'Quach Cong Lich\xa0',
 'Edgardo\xa0Alejan',
 'Luong Van Thao\xa0

In [436]:
namess=pd.DataFrame(names)

In [437]:
namess.to_csv('names.csv', encoding='utf-8')

In [422]:
temp_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Su Bingtian,CHN,9.92,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,29 Aug 89,« »,,[]
1,2,Tosin Ogunode,QAT,10.00,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,2 Mar 94,PB »,,[]
2,3,Ryota Yamagata,JPN,10.00,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,10 Jun 92,=PB »,,[]
3,4,Abdullah Abkar Mohammed,KSA,10.10,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,1 Jun 97,« »,,[]
4,5,Yang Chun-Han,TPE,10.17,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,1 Jan 97,« »,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,6,Meg Hemphill,JPN,43.71,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,23 May 96,« »,,"{0: ' ', 1: ' 43.71 39.89 43.19..."
1142,7,Chu Chia-Ling,TPE,42.74,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,11 Jan 91,SB «,,"{0: ' ', 1: ' 39.39 42.28 SB 42...."
1143,8,Wang Qingling,CHN,39.33,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,14 Jan 93,« »,,"{0: ' ', 1: ' 35.60 38.21 39.33..."
1144,9,Jeong Yeon-Jin,KOR,35.64,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,25 Mar 92,« »,,"{0: ' ', 1: ' 35.64 33.09 31.26..."


In [423]:
columns =  list(temp_df.columns) 
    
for col in columns:
    temp_df[col] = temp_df[col].astype(str)
    temp_df[col] = temp_df[col].str.replace('\xa0', ' ', regex=True)
    temp_df[col] = temp_df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    
    

In [439]:
temp_df.to_csv("2015_SEAG.csv", encoding='utf-8')

In [394]:
asian_games_2014 = temp_df

In [412]:
asian_games_2023 = temp_df

In [425]:
asian_games_2018 = temp_df

In [426]:
asian_games_2014

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Femi Ogunode,QAT,9.93,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,15 May 91,PB »,0.184,[]
1,2,Su Bingtian,CHN,10.10,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,29 Aug 89,SB «,0.145,[]
2,3,Kei Takase,JPN,10.15,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,25 Nov 88,« »,0.152,[]
3,4,Zhang Peimeng,CHN,10.18,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,13 Mar 87,« »,0.128,[]
4,5,Reza Ghasemi,IRI,10.25,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,24 Jul 87,SB «,0.161,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2,Huang Yu-Ting,TPE,14.36,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,17 Nov 88,« »,0.239,[]
931,3,Kotchakorn Khamrueangsri,THA,14.52,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,13 Mar 95,« »,0.222,[]
932,4,Sushmita Singha Roy,IND,14.67,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,26 Mar 84,« »,0.229,[]
933,5,Yekaterina Voronina,UZB,15.19,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,16 Feb 92,PB »,0.192,[]


In [427]:
results = pd.concat([asian_games_2014, asian_games_2018], axis=0)

In [428]:
results = pd.concat([results, asian_games_2023], axis=0)

In [429]:
results

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Femi Ogunode,QAT,9.93,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,15 May 91,PB »,0.184,[]
1,2,Su Bingtian,CHN,10.10,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,29 Aug 89,SB «,0.145,[]
2,3,Kei Takase,JPN,10.15,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,25 Nov 88,« »,0.152,[]
3,4,Zhang Peimeng,CHN,10.18,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,13 Mar 87,« »,0.128,[]
4,5,Reza Ghasemi,IRI,10.25,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,24 Jul 87,SB «,0.161,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,3,,SRI,3:30.88,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Nadeesha Ramanayake, M.H.Jaye..."
849,4,,VIE,3:31.61,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Nguyen Thi Ngoc, Hoang Thi Mi..."
850,5,,PHI,3:40.78,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Robyn Brown, Angel Frank, Mau..."
851,6,,THA,3:50.44,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont..."


In [430]:
results.to_csv('results.csv', encoding='utf-8')